In [1]:
import os
import scanpy
import anndata
import scanpy.api as sc
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import re
import sklearn
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from umi_tools import umi_methods
from collections import Counter
import logging as logg
import random
import seaborn
import sys
import shutil

#Load my pipeline functions
import importlib.util
spec = importlib.util.spec_from_file_location("ScanpyUtilsMT", os.path.expanduser("~/ScanpyUtilsMT.py"))
sc_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(sc_utils)
#Get lots of feedback from scanpy
logg.basicConfig(level=logg.INFO)
sc.settings.verbosity=5

refname="refdata-celranger-mmul8-toplevel"

sc.settings.autosave=True
sc.settings.autoshow=False


/home/mt/code/utils/miniconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/mt/code/utils/miniconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/mt/code/utils/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/mt/code/utils/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mt/code/utils/miniconda3/lib/python3.6/runpy.py", lin

FileNotFoundError: [Errno 2] No such file or directory: '/home/mt/ScanpyUtilsMT.py'

In [55]:
basepath=os.path.expanduser("/scrapp2/mtschmitz/")
adata=sc.read(os.path.join(basepath,'macaqueDevBrain-concat_files-MICROGLIA.h5ad'))
sc.settings.figdir="/scrapp2/mtschmitz/Basic_600_4_4_Microglia/"

In [58]:
print([x in adata.var.index for x in ['UTY','SRY','KDM5D','DDX3Y','ZFY']])
pd.DataFrame(adata[:,['UTY','SRY','KDM5D','DDX3Y']].X.todense(),columns=['UTY','SRY','KDM5D','DDX3Y'],index=adata.obs.index)
adata.obs.join(pd.DataFrame(adata[:,['UTY','SRY','KDM5D','DDX3Y']].X.todense(),columns=['UTY','SRY','KDM5D','DDX3Y'],index=adata.obs.index))

[True, True, True, True, False, True]


,n_genes,n_counts,percent_ribo,percent_mito,batch,timepoint,region,UTY,SRY,KDM5D,DDX3Y
index,,,,,,,,,,,
AAGCCGCGTCTTCGTC-1-0,670,1914.0,0.332811,0.011494,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACATACGGTCGGCATC-1-0,483,725.0,0.073103,0.122759,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACATGGTCAAACTGCT-1-0,775,1490.0,0.186577,0.040940,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACATGGTTCAGAGACG-1-0,396,750.0,0.264000,0.018667,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACCAGTAGTGTTCTTT-1-0,977,2056.0,0.214494,0.014591,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACGCCGACATTAACCG-1-0,518,1083.0,0.268698,0.048938,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACGGGCTCATCACCCT-1-0,380,888.0,0.314189,0.088964,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACGGGTCCAAACAACA-1-0,597,1405.0,0.278292,0.059786,0,40.0,hindbrain,0.0,0.0,0.0,0.0
ACTGAACGTCTCTCGT-1-0,494,750.0,0.164000,0.040000,0,40.0,hindbrain,0.0,0.0,0.0,0.0


In [60]:
adata.obs=adata.obs.join(pd.DataFrame(adata[:,['UTY','SRY','KDM5D','DDX3Y','ZFY']].X.todense(),columns=['UTY','SRY','KDM5D','DDX3Y','ZFY'],index=adata.obs.index))
#adata.obs[['UTY','SRY','KDM5D','DDX3Y']]=adata[:,['UTY','SRY','KDM5D','DDX3Y']].X.todense()

In [62]:
adata.obs=adata.obs.join(pd.DataFrame(adata[:,['UTY','SRY','KDM5D','DDX3Y','ZFY']].X.todense(),columns=['UTY','SRY','KDM5D','DDX3Y','ZFY'],index=adata.obs.index))
sc.pl.violin(adata,groupby='timepoint',keys=['UTY','SRY','KDM5D','DDX3Y','ZFY'])

saving figure to file /scrapp2/mtschmitz/Basic_600_4_4_Microglia/violin.pdf


In [68]:
adata.obs.groupby('timepoint').sum()[['UTY','SRY','KDM5D','DDX3Y','ZFY']]

,UTY,SRY,KDM5D,DDX3Y,ZFY
timepoint,,,,,
40.0,1.0,0.0,0.0,21.0,5.0
50.0,0.0,0.0,0.0,0.0,0.0
65.0,0.0,0.0,0.0,0.0,0.0
80.0,3.0,0.0,4.0,44.0,16.0
90.0,3.0,0.0,7.0,39.0,5.0
100.0,105.0,0.0,99.0,790.0,101.0


In [ ]:
sc.pp.filter_genes(adata,min_counts=1)
adata=sc_utils.std_norm_transform(adata,n_top_genes=8000,tsne=True,save=True)


filtered out 2582 genes that are detected in less than 1 counts
running recipe zheng17
    normalizing by total count per cell
        finished (0:00:01.84): normalized adata.X and added
        'n_counts_all', counts per cell before normalization (adata.obs)
    extracting highly variable genes
    the 8000 top genes correspond to a normalized dispersion cutoff of
        finished (0:00:00.86)
    normalizing by total count per cell


In [ ]:
if 'X_tsne' in adata.obsm.keys():
    sc.pl.tsne(adata,color='batch',save='_batchcolor_MGonly')
    sc.pl.tsne(adata,color='region',save='_regioncolor_MGonly')
    sc.pl.tsne(adata,color='timepoint',save='_timepointcolor_MGonly')

sc.pl.umap(adata,color='batch',save='_batchcolor_MGonly')
sc.pl.umap(adata,color='region',save='_regioncolor_MGonly')
sc.pl.umap(adata,color='timepoint',save='_timepointcolor_MGonly')


In [ ]:
adata=sc_utils.cell_cycle_score(adata)
adata=sc_utils.log_reg_diff_exp(adata)
adata=sc_utils.marker_analysis(adata,markerpath="~/markers/Markers.txt")

In [ ]:
sc.pl.violin(adata,groupby='batch',keys='n_counts',save='ncount')

In [ ]:
sc.pp.regress_out(adata,keys=['batch'])
sc.pp.pca(adata)
sc.tl.tsne(adata)
sc.pl.tsne(adata,color='batch',save='_batchcolor_MGonly_batchregressed')
sc.pl.tsne(adata,color='region',save='_regioncolor_MGonly_batchregressed')
sc.pl.tsne(adata,color='timepoint',save='_timepointcolor_MGonly_batchregressed')


In [ ]:
sc.tl.phate
sc.pl.phate